In [1]:
import pandas as pd
import numpy as np

In [2]:
MERIDIAN_PATH = "../../data/life/Bite Off More Than You Can Chew Braces profile - Sheet1.csv"
TEETH_UP_PATH = "../../data/life/BOMTYCC_teeth_up_1854211842023.json"
TEETH_DOWN_PATH = "../../data/life/BOMTYCC_teeth_down_1826391842023.json"

In [3]:
teeth_up = pd.read_json(TEETH_UP_PATH)
teeth_down = pd.read_json(TEETH_DOWN_PATH)
teeth_df = pd.concat([teeth_up, teeth_down])
teeth_df = teeth_df.reset_index(drop=True)

In [4]:
meridian_df = pd.read_csv(MERIDIAN_PATH)
meridian_df.dropna(subset=["Tooth number", "Domain"], inplace=True)
f = lambda s: [int(s) for s in s.split("-")]
meridian_df["Tooth number"] = meridian_df["Tooth number"].apply(f)

In [62]:
# create the training dataset
train_df = pd.DataFrame() 

n_teeth = teeth_df["data"].shape[0]
# the default position of irrelevant teeth
non_tooth = [-1.0, -1.0]

# loop through meridian rows
for row_index, row in meridian_df.iterrows():
    # the ids of the associated teeth
    teeth_ids = row["Tooth number"]
    # filter out the irrelevant teeth data
    mask = teeth_df["data"].apply(lambda x: x['toothId'] in teeth_ids)
    associated_teeth = teeth_df["data"][mask]
    # get indexes of the associated teeth in the context of all teeth
    mask = teeth_data.isin(associated_teeth)
    # flatnonzero returns indexes of values > 0
    associated_teeth_indexes = np.flatnonzero(mask)
    # filter columns with domains ratings
    # There's ratings associated with each dental brace gin wore in the past    
    meridian_life_ratings = row.filter(like='Brace').dropna()
    n_ratings = meridian_life_ratings.shape[0]

    xys = []
    
    for rating_index in range(n_ratings):
        # set positions of all teeth teeth to the non teeth value 
        x = np.full((n_teeth, len(non_tooth)), non_tooth)
        y = meridian_life_ratings.iloc[rating_index]
        # update x values where there's associated teeth
        for tup in zip(associated_teeth, associated_teeth_indexes):
            (tooth, tooth_index) = tup
            # print(row["Domain"])
            #print(meridian_life_ratings.iloc[rating_index])
            x[tooth_index] = tooth["toothPositions"][rating_index]
        xys.append([x,y])
        
    train_df = train_df.append(pd.Series(xys), ignore_index=True)

# next check next week if the df has the desired col0: [x1,y1], col1: [x2,y2] structure

0     [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
1     [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
2     [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
3     [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
4     [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
                            ...                        
94    [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
95    [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
96    [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
97    [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
98    [[[-1.0, -1.0], [-1.0, -1.0], [-1.0, -1.0], [-...
Name: 0, Length: 99, dtype: object